# Prep

Setting up some prior functionality

In [2]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.5.1+cu101 True


# Load a model

First we have to decide if our model should be pretrained. 

This greatly depends on the size of a dataset. Smaller datasets rely more on finetuning. 

In [3]:
pretrained = True

if pretrained:
    # Get pretrained weights
    checkpoint = torch.hub.load_state_dict_from_url(
                url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
                map_location='cpu',
                check_hash=True)

    # Remove class weights
    del checkpoint["model"]["class_embed.weight"]
    del checkpoint["model"]["class_embed.bias"]

    # SaveOGH
    torch.save(checkpoint,
               'detr-r50_no-class-head.pth')

# Dataset

Our dataset should be loadable as a COCO format

This allows us to use the pycocotools to load the data dict for the main python script

In [4]:
# alternatively, implement your own coco-type dataset loader in datasets and add this "key" to datasets/__init__.py
dataset_file = "coco_petct" 

# should lead to a directory with a train2017 and val2017 folder as well as an annotations folder
#dataDir='/COCO_dataset/' 
#dataDir = '/media/storage/Joy/datasets/DETR_MIP/FDG-PET-CT-Lesions/'
# dataDir = '/gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/'
dataDir = '/gpfs/fs0/data/stanford_data/petct/DETR_MIP2/FDG-PET-CT-Lesions/'

# this int should be the actual number of classes + 1 (for no class)
num_classes = 2

outDir = 'outputs'
resume = "detr-r50_no-class-head.pth" if pretrained else ""

In [5]:
!ls /gpfs/fs0/data/stanford_data/petct/DETR_MIP2/FDG-PET-CT-Lesions/annotations/

detr_test.json	detr_train.json  detr_val.json


In [6]:
!ls $dataDir

annotations  images


In [7]:
data_list = [0,1,2,3,4,5,6,7,8]
new_list = [data_list[i:i+3] for i in range(0, len(data_list), 3)]
new_list

list(range(0,5))

import random
 
# initializing lists
# test_list1 = [6, 4, 8, 9, 10]
# test_list2 = [1, 2, 3, 4, 5]
test_list1 = [{'id':1},{'id':2},{'id':3}]
test_list2 = [{'id':1},{'id':2},{'id':3}]
 
# printing lists
print(f"The original list 1 : {test_list1}")
print(f"The original list 2 : {test_list2}")
 
# Shuffle two lists with same order
# Using zip() + * operator + shuffle()
temp = list(zip(test_list1, test_list2))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
res1, res2 = list(res1), list(res2)
 
# Printing result
print(f"List 1 after shuffle :  {res1}")
print(f"List 2 after shuffle :  {res2}")

The original list 1 : [{'id': 1}, {'id': 2}, {'id': 3}]
The original list 2 : [{'id': 1}, {'id': 2}, {'id': 3}]
List 1 after shuffle :  [{'id': 1}, {'id': 2}, {'id': 3}]
List 2 after shuffle :  [{'id': 1}, {'id': 2}, {'id': 3}]


# Training

We use the main.py script to run our training

In [ ]:
# !python main_petct.py \
#   --dataset_file coco_petct \
#   --coco_petct_path /gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/ \
#   --output_dir /home/joywu/detr/output \
#   --resume detr-r50_no-class-head.pth \
#   --num_classes 2 \
#   --lr 1e-5 \
#   --lr_backbone 1e-6 \
#   --epochs 1

# changed numpy from 1.22.4 to 1.20.0 to 1.21.6 still didn't work so updated scipy and pycocotools to latest

In [74]:
!/home/joywu/anaconda3/envs/detr_py38/bin/python main_petct.py \
  --dataset_file $dataset_file \
  --coco_petct_path $dataDir \
  --output_dir $outDir \
  --resume $resume \
  --num_classes $num_classes \
  --lr 1e-5 \
  --lr_backbone 1e-6 \
  --epochs 1

Not using distributed mode
git:
  sha: 43e6680020a7b7365aeb5b88f6469c1a7b10721c, status: has uncommited changes, branch: master

Namespace(aux_loss=True, backbone='resnet50', batch_size=2, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path=None, coco_path=None, coco_petct_path='/gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/', cross_val=False, dataset_file='coco_petct', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=1, eval=False, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=1e-05, lr_backbone=1e-06, lr_drop=200, mask_loss_coef=1, masks=False, nheads=8, num_classes=2, num_queries=100, num_workers=2, output_dir='outputs', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='detr-r50_no-class-head.pth', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, train_split=0.9, val_fold=0, weig

Epoch: [0]  [   40/19488]  eta: 1:00:03  lr: 0.000010  class_error: 12.50  loss: 23.9193 (29.1009)  loss_ce: 0.9464 (1.0900)  loss_bbox: 0.9120 (1.5669)  loss_giou: 2.0000 (2.1449)  loss_ce_0: 1.0809 (1.1093)  loss_bbox_0: 1.0640 (1.5899)  loss_giou_0: 2.0233 (2.1824)  loss_ce_1: 1.0686 (1.1370)  loss_bbox_1: 1.0372 (1.6007)  loss_giou_1: 2.0000 (2.1607)  loss_ce_2: 1.0194 (1.1416)  loss_bbox_2: 0.8658 (1.5708)  loss_giou_2: 2.0071 (2.1513)  loss_ce_3: 1.0023 (1.1417)  loss_bbox_3: 0.9282 (1.5803)  loss_giou_3: 2.0000 (2.1134)  loss_ce_4: 0.9830 (1.1263)  loss_bbox_4: 0.8877 (1.5504)  loss_giou_4: 2.0000 (2.1431)  loss_ce_unscaled: 0.9464 (1.0900)  class_error_unscaled: 7.1429 (12.4885)  loss_bbox_unscaled: 0.1824 (0.3134)  loss_giou_unscaled: 1.0000 (1.0724)  cardinality_error_unscaled: 67.5000 (78.6707)  loss_ce_0_unscaled: 1.0809 (1.1093)  loss_bbox_0_unscaled: 0.2128 (0.3180)  loss_giou_0_unscaled: 1.0116 (1.0912)  cardinality_error_0_unscaled: 86.5000 (89.1098)  loss_ce_1_unscaled

Epoch: [0]  [   90/19488]  eta: 0:58:44  lr: 0.000010  class_error: 100.00  loss: 18.9638 (24.7347)  loss_ce: 0.5837 (0.8676)  loss_bbox: 0.4032 (1.0699)  loss_giou: 1.9822 (2.0836)  loss_ce_0: 0.7602 (0.9528)  loss_bbox_0: 0.4959 (1.1743)  loss_giou_0: 2.0000 (2.1382)  loss_ce_1: 0.7381 (0.9602)  loss_bbox_1: 0.5144 (1.1540)  loss_giou_1: 2.0000 (2.1060)  loss_ce_2: 0.7088 (0.9422)  loss_bbox_2: 0.4349 (1.0980)  loss_giou_2: 1.9787 (2.0783)  loss_ce_3: 0.6296 (0.9152)  loss_bbox_3: 0.4102 (1.0851)  loss_giou_3: 1.9755 (2.0658)  loss_ce_4: 0.6282 (0.9028)  loss_bbox_4: 0.3723 (1.0594)  loss_giou_4: 2.0000 (2.0813)  loss_ce_unscaled: 0.5837 (0.8676)  class_error_unscaled: 100.0000 (48.0416)  loss_bbox_unscaled: 0.0806 (0.2140)  loss_giou_unscaled: 0.9911 (1.0418)  cardinality_error_unscaled: 3.5000 (42.1484)  loss_ce_0_unscaled: 0.7602 (0.9528)  loss_bbox_0_unscaled: 0.0992 (0.2349)  loss_giou_0_unscaled: 1.0000 (1.0691)  cardinality_error_0_unscaled: 40.5000 (69.8681)  loss_ce_1_unscal

Epoch: [0]  [  140/19488]  eta: 0:58:26  lr: 0.000010  class_error: 100.00  loss: 17.0463 (22.5952)  loss_ce: 0.5290 (0.7874)  loss_bbox: 0.3469 (0.8458)  loss_giou: 1.9740 (2.0491)  loss_ce_0: 0.6147 (0.8583)  loss_bbox_0: 0.4899 (0.9548)  loss_giou_0: 2.0000 (2.1012)  loss_ce_1: 0.6058 (0.8599)  loss_bbox_1: 0.4552 (0.9217)  loss_giou_1: 1.9607 (2.0675)  loss_ce_2: 0.5651 (0.8403)  loss_bbox_2: 0.3676 (0.8743)  loss_giou_2: 1.9481 (2.0399)  loss_ce_3: 0.5528 (0.8157)  loss_bbox_3: 0.3523 (0.8585)  loss_giou_3: 1.9506 (2.0277)  loss_ce_4: 0.5476 (0.8077)  loss_bbox_4: 0.3707 (0.8397)  loss_giou_4: 1.9674 (2.0458)  loss_ce_unscaled: 0.5290 (0.7874)  class_error_unscaled: 100.0000 (65.5091)  loss_bbox_unscaled: 0.0694 (0.1692)  loss_giou_unscaled: 0.9870 (1.0245)  cardinality_error_unscaled: 2.5000 (29.0957)  loss_ce_0_unscaled: 0.6147 (0.8583)  loss_bbox_0_unscaled: 0.0980 (0.1910)  loss_giou_0_unscaled: 1.0000 (1.0506)  cardinality_error_0_unscaled: 12.0000 (51.9007)  loss_ce_1_unscal

Epoch: [0]  [  190/19488]  eta: 0:58:13  lr: 0.000010  class_error: 100.00  loss: 15.9638 (21.4541)  loss_ce: 0.4414 (0.7424)  loss_bbox: 0.2256 (0.7320)  loss_giou: 1.8660 (2.0242)  loss_ce_0: 0.5501 (0.7974)  loss_bbox_0: 0.2996 (0.8360)  loss_giou_0: 1.9533 (2.0736)  loss_ce_1: 0.5459 (0.7999)  loss_bbox_1: 0.3234 (0.8101)  loss_giou_1: 1.8920 (2.0426)  loss_ce_2: 0.5028 (0.7820)  loss_bbox_2: 0.2750 (0.7704)  loss_giou_2: 1.8607 (2.0159)  loss_ce_3: 0.4709 (0.7604)  loss_bbox_3: 0.2852 (0.7507)  loss_giou_3: 1.8898 (2.0080)  loss_ce_4: 0.4508 (0.7544)  loss_bbox_4: 0.2691 (0.7308)  loss_giou_4: 1.8425 (2.0234)  loss_ce_unscaled: 0.4414 (0.7424)  class_error_unscaled: 100.0000 (74.0999)  loss_bbox_unscaled: 0.0451 (0.1464)  loss_giou_unscaled: 0.9330 (1.0121)  cardinality_error_unscaled: 2.5000 (22.7984)  loss_ce_0_unscaled: 0.5501 (0.7974)  loss_bbox_0_unscaled: 0.0599 (0.1672)  loss_giou_0_unscaled: 0.9766 (1.0368)  cardinality_error_0_unscaled: 6.5000 (41.1073)  loss_ce_1_unscale

...

Epoch: [0]  [19290/19488]  eta: 0:00:35  lr: 0.000010  class_error: 75.00  loss: 12.3993 (13.8000)  loss_ce: 0.2531 (0.4281)  loss_bbox: 0.1167 (0.1590)  loss_giou: 1.7502 (1.7062)  loss_ce_0: 0.2663 (0.4089)  loss_bbox_0: 0.1569 (0.1729)  loss_giou_0: 1.7508 (1.7396)  loss_ce_1: 0.2946 (0.4325)  loss_bbox_1: 0.1264 (0.1585)  loss_giou_1: 1.7189 (1.7153)  loss_ce_2: 0.2636 (0.4288)  loss_bbox_2: 0.1326 (0.1583)  loss_giou_2: 1.7387 (1.7116)  loss_ce_3: 0.2550 (0.4246)  loss_bbox_3: 0.1151 (0.1591)  loss_giou_3: 1.6487 (1.7065)  loss_ce_4: 0.2495 (0.4249)  loss_bbox_4: 0.1091 (0.1596)  loss_giou_4: 1.7274 (1.7057)  loss_ce_unscaled: 0.2531 (0.4281)  class_error_unscaled: 23.0769 (34.4868)  loss_bbox_unscaled: 0.0233 (0.0318)  loss_giou_unscaled: 0.8751 (0.8531)  cardinality_error_unscaled: 1.0000 (7.3455)  loss_ce_0_unscaled: 0.2663 (0.4089)  loss_bbox_0_unscaled: 0.0314 (0.0346)  loss_giou_0_unscaled: 0.8754 (0.8698)  cardinality_error_0_unscaled: 2.0000 (7.9499)  loss_ce_1_unscaled: 0

Epoch: [0]  [19340/19488]  eta: 0:00:26  lr: 0.000010  class_error: 50.00  loss: 12.0000 (13.7988)  loss_ce: 0.3764 (0.4281)  loss_bbox: 0.1161 (0.1589)  loss_giou: 1.5947 (1.7060)  loss_ce_0: 0.3487 (0.4089)  loss_bbox_0: 0.1672 (0.1728)  loss_giou_0: 1.6869 (1.7395)  loss_ce_1: 0.3480 (0.4325)  loss_bbox_1: 0.0943 (0.1584)  loss_giou_1: 1.5541 (1.7152)  loss_ce_2: 0.3689 (0.4288)  loss_bbox_2: 0.0990 (0.1582)  loss_giou_2: 1.6022 (1.7115)  loss_ce_3: 0.3178 (0.4246)  loss_bbox_3: 0.1131 (0.1590)  loss_giou_3: 1.5740 (1.7064)  loss_ce_4: 0.3587 (0.4249)  loss_bbox_4: 0.0881 (0.1595)  loss_giou_4: 1.6159 (1.7055)  loss_ce_unscaled: 0.3764 (0.4281)  class_error_unscaled: 28.5714 (34.4774)  loss_bbox_unscaled: 0.0232 (0.0318)  loss_giou_unscaled: 0.7974 (0.8530)  cardinality_error_unscaled: 0.5000 (7.3463)  loss_ce_0_unscaled: 0.3487 (0.4089)  loss_bbox_0_unscaled: 0.0334 (0.0346)  loss_giou_0_unscaled: 0.8434 (0.8697)  cardinality_error_0_unscaled: 1.0000 (7.9491)  loss_ce_1_unscaled: 0

Epoch: [0]  [19390/19488]  eta: 0:00:17  lr: 0.000010  class_error: 0.00  loss: 12.0000 (13.7960)  loss_ce: 0.2173 (0.4279)  loss_bbox: 0.1223 (0.1588)  loss_giou: 1.6510 (1.7058)  loss_ce_0: 0.2374 (0.4088)  loss_bbox_0: 0.0964 (0.1727)  loss_giou_0: 1.5133 (1.7393)  loss_ce_1: 0.2382 (0.4323)  loss_bbox_1: 0.0955 (0.1584)  loss_giou_1: 1.5268 (1.7150)  loss_ce_2: 0.2118 (0.4286)  loss_bbox_2: 0.1182 (0.1581)  loss_giou_2: 1.5683 (1.7112)  loss_ce_3: 0.2214 (0.4245)  loss_bbox_3: 0.1045 (0.1589)  loss_giou_3: 1.5581 (1.7062)  loss_ce_4: 0.2180 (0.4247)  loss_bbox_4: 0.1168 (0.1594)  loss_giou_4: 1.5790 (1.7053)  loss_ce_unscaled: 0.2173 (0.4279)  class_error_unscaled: 0.0000 (34.4495)  loss_bbox_unscaled: 0.0245 (0.0318)  loss_giou_unscaled: 0.8255 (0.8529)  cardinality_error_unscaled: 3.0000 (7.3464)  loss_ce_0_unscaled: 0.2374 (0.4088)  loss_bbox_0_unscaled: 0.0193 (0.0345)  loss_giou_0_unscaled: 0.7567 (0.8696)  cardinality_error_0_unscaled: 5.0000 (7.9488)  loss_ce_1_unscaled: 0.2

Epoch: [0]  [19440/19488]  eta: 0:00:08  lr: 0.000010  class_error: 75.00  loss: 13.1679 (13.7943)  loss_ce: 0.4462 (0.4279)  loss_bbox: 0.1847 (0.1587)  loss_giou: 1.4312 (1.7057)  loss_ce_0: 0.4207 (0.4088)  loss_bbox_0: 0.1718 (0.1726)  loss_giou_0: 1.5486 (1.7391)  loss_ce_1: 0.4197 (0.4323)  loss_bbox_1: 0.1717 (0.1583)  loss_giou_1: 1.5317 (1.7148)  loss_ce_2: 0.4194 (0.4286)  loss_bbox_2: 0.1756 (0.1580)  loss_giou_2: 1.5713 (1.7111)  loss_ce_3: 0.4666 (0.4245)  loss_bbox_3: 0.1738 (0.1588)  loss_giou_3: 1.5276 (1.7060)  loss_ce_4: 0.4319 (0.4247)  loss_bbox_4: 0.1684 (0.1593)  loss_giou_4: 1.4145 (1.7052)  loss_ce_unscaled: 0.4462 (0.4279)  class_error_unscaled: 35.0000 (34.4514)  loss_bbox_unscaled: 0.0369 (0.0317)  loss_giou_unscaled: 0.7156 (0.8528)  cardinality_error_unscaled: 2.5000 (7.3449)  loss_ce_0_unscaled: 0.4207 (0.4088)  loss_bbox_0_unscaled: 0.0344 (0.0345)  loss_giou_0_unscaled: 0.7743 (0.8696)  cardinality_error_0_unscaled: 2.5000 (7.9477)  loss_ce_1_unscaled: 0

Epoch: [0]  [19487/19488]  eta: 0:00:00  lr: 0.000010  class_error: 0.00  loss: 12.0000 (13.7905)  loss_ce: 0.2445 (0.4276)  loss_bbox: 0.0878 (0.1586)  loss_giou: 1.5886 (1.7054)  loss_ce_0: 0.2580 (0.4086)  loss_bbox_0: 0.0973 (0.1725)  loss_giou_0: 1.5951 (1.7389)  loss_ce_1: 0.2528 (0.4319)  loss_bbox_1: 0.0780 (0.1582)  loss_giou_1: 1.5379 (1.7146)  loss_ce_2: 0.2517 (0.4283)  loss_bbox_2: 0.0866 (0.1579)  loss_giou_2: 1.6237 (1.7108)  loss_ce_3: 0.2443 (0.4242)  loss_bbox_3: 0.0786 (0.1587)  loss_giou_3: 1.5872 (1.7058)  loss_ce_4: 0.2443 (0.4244)  loss_bbox_4: 0.0968 (0.1592)  loss_giou_4: 1.5650 (1.7049)  loss_ce_unscaled: 0.2445 (0.4276)  class_error_unscaled: 27.2727 (34.4307)  loss_bbox_unscaled: 0.0176 (0.0317)  loss_giou_unscaled: 0.7943 (0.8527)  cardinality_error_unscaled: 1.0000 (7.3424)  loss_ce_0_unscaled: 0.2580 (0.4086)  loss_bbox_0_unscaled: 0.0195 (0.0345)  loss_giou_0_unscaled: 0.7976 (0.8694)  cardinality_error_0_unscaled: 2.5000 (7.9477)  loss_ce_1_unscaled: 0.

In [8]:
!/home/joywu/anaconda3/envs/detr_py38/bin/python main_petct.py \
  --dataset_file $dataset_file \
  --coco_petct_path $dataDir \
  --output_dir $outDir \
  --resume $resume \
  --num_classes $num_classes \
  --lr 1e-5 \
  --lr_backbone 1e-6 \
  --epochs 1 

Not using distributed mode
git:
  sha: c26a1bfa7f93d39cf26858bac4d10b284e5188c0, status: has uncommited changes, branch: master

Namespace(aux_loss=True, backbone='resnet50', batch_size=2, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path=None, coco_path=None, coco_petct_path='/gpfs/fs0/data/stanford_data/petct/DETR_MIP2/FDG-PET-CT-Lesions/', cross_val=False, dataset_file='coco_petct', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=1, eval=False, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=1e-05, lr_backbone=1e-06, lr_drop=200, mask_loss_coef=1, masks=False, nheads=8, num_classes=2, num_queries=100, num_workers=2, output_dir='outputs', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='detr-r50_no-class-head.pth', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, train_split=0.9, val_fold=0, wei

Test:  [  50/2184]  eta: 0:04:04  class_error: 100.00  loss: 12.3711 (24.9124)  loss_ce: 2.0749 (1.6161)  loss_bbox: 0.0000 (1.4704)  loss_giou: 0.0000 (1.1431)  loss_ce_0: 1.9618 (1.4379)  loss_bbox_0: 0.0000 (1.4379)  loss_giou_0: 0.0000 (1.1283)  loss_ce_1: 2.0791 (1.5511)  loss_bbox_1: 0.0000 (1.4045)  loss_giou_1: 0.0000 (1.1359)  loss_ce_2: 2.0870 (1.5791)  loss_bbox_2: 0.0000 (1.4316)  loss_giou_2: 0.0000 (1.1450)  loss_ce_3: 2.1220 (1.6359)  loss_bbox_3: 0.0000 (1.4520)  loss_giou_3: 0.0000 (1.1371)  loss_ce_4: 2.0839 (1.6223)  loss_bbox_4: 0.0000 (1.4510)  loss_giou_4: 0.0000 (1.1331)  loss_ce_unscaled: 2.0749 (1.6161)  class_error_unscaled: 100.0000 (52.9412)  loss_bbox_unscaled: 0.0000 (0.2941)  loss_giou_unscaled: 0.0000 (0.5715)  cardinality_error_unscaled: 100.0000 (93.6471)  loss_ce_0_unscaled: 1.9618 (1.4379)  loss_bbox_0_unscaled: 0.0000 (0.2876)  loss_giou_0_unscaled: 0.0000 (0.5641)  cardinality_error_0_unscaled: 100.0000 (93.6471)  loss_ce_1_unscaled: 2.0791 (1.5511

Test:  [ 100/2184]  eta: 0:03:06  class_error: 100.00  loss: 11.0850 (18.2559)  loss_ce: 1.8782 (1.7954)  loss_bbox: 0.0000 (0.7425)  loss_giou: 0.0000 (0.5772)  loss_ce_0: 1.6633 (1.5636)  loss_bbox_0: 0.0000 (0.7261)  loss_giou_0: 0.0000 (0.5697)  loss_ce_1: 1.8368 (1.7077)  loss_bbox_1: 0.0000 (0.7092)  loss_giou_1: 0.0000 (0.5736)  loss_ce_2: 1.9082 (1.7435)  loss_bbox_2: 0.0000 (0.7229)  loss_giou_2: 0.0000 (0.5782)  loss_ce_3: 1.9681 (1.8285)  loss_bbox_3: 0.0000 (0.7332)  loss_giou_3: 0.0000 (0.5742)  loss_ce_4: 1.9431 (1.8056)  loss_bbox_4: 0.0000 (0.7327)  loss_giou_4: 0.0000 (0.5721)  loss_ce_unscaled: 1.8782 (1.7954)  class_error_unscaled: 100.0000 (76.2376)  loss_bbox_unscaled: 0.0000 (0.1485)  loss_giou_unscaled: 0.0000 (0.2886)  cardinality_error_unscaled: 100.0000 (96.7921)  loss_ce_0_unscaled: 1.6633 (1.5636)  loss_bbox_0_unscaled: 0.0000 (0.1452)  loss_giou_0_unscaled: 0.0000 (0.2849)  cardinality_error_0_unscaled: 100.0000 (96.7921)  loss_ce_1_unscaled: 1.8368 (1.7077

Test:  [ 150/2184]  eta: 0:02:49  class_error: 0.00  loss: 11.5350 (17.0623)  loss_ce: 1.7627 (1.8025)  loss_bbox: 0.0000 (0.6279)  loss_giou: 0.0000 (0.4861)  loss_ce_0: 1.4815 (1.5575)  loss_bbox_0: 0.0000 (0.6354)  loss_giou_0: 0.0000 (0.4792)  loss_ce_1: 1.5675 (1.6801)  loss_bbox_1: 0.0000 (0.6156)  loss_giou_1: 0.0000 (0.4815)  loss_ce_2: 1.6513 (1.7391)  loss_bbox_2: 0.0000 (0.6128)  loss_giou_2: 0.0000 (0.4860)  loss_ce_3: 1.8590 (1.8401)  loss_bbox_3: 0.0000 (0.6198)  loss_giou_3: 0.0000 (0.4833)  loss_ce_4: 1.8229 (1.8161)  loss_bbox_4: 0.0000 (0.6199)  loss_giou_4: 0.0000 (0.4794)  loss_ce_unscaled: 1.7627 (1.8025)  class_error_unscaled: 100.0000 (79.4702)  loss_bbox_unscaled: 0.0000 (0.1256)  loss_giou_unscaled: 0.0000 (0.2430)  cardinality_error_unscaled: 100.0000 (97.6159)  loss_ce_0_unscaled: 1.4815 (1.5575)  loss_bbox_0_unscaled: 0.0000 (0.1271)  loss_giou_0_unscaled: 0.0000 (0.2396)  cardinality_error_0_unscaled: 100.0000 (97.6225)  loss_ce_1_unscaled: 1.5675 (1.6801) 

Test:  [ 200/2184]  eta: 0:02:45  class_error: 0.00  loss: 12.0533 (18.9942)  loss_ce: 1.5047 (1.7244)  loss_bbox: 0.0000 (0.8733)  loss_giou: 0.0000 (0.6332)  loss_ce_0: 1.3010 (1.4844)  loss_bbox_0: 0.0000 (0.9276)  loss_giou_0: 0.0000 (0.6236)  loss_ce_1: 1.3571 (1.5956)  loss_bbox_1: 0.0000 (0.8805)  loss_giou_1: 0.0000 (0.6277)  loss_ce_2: 1.3780 (1.6629)  loss_bbox_2: 0.0000 (0.8616)  loss_giou_2: 0.0000 (0.6302)  loss_ce_3: 1.4765 (1.7517)  loss_bbox_3: 0.0000 (0.8664)  loss_giou_3: 0.0000 (0.6284)  loss_ce_4: 1.5666 (1.7343)  loss_bbox_4: 0.0000 (0.8604)  loss_giou_4: 0.0000 (0.6279)  loss_ce_unscaled: 1.5047 (1.7244)  class_error_unscaled: 100.0000 (71.6418)  loss_bbox_unscaled: 0.0000 (0.1747)  loss_giou_unscaled: 0.0000 (0.3166)  cardinality_error_unscaled: 100.0000 (96.9876)  loss_ce_0_unscaled: 1.3010 (1.4844)  loss_bbox_0_unscaled: 0.0000 (0.1855)  loss_giou_0_unscaled: 0.0000 (0.3118)  cardinality_error_0_unscaled: 100.0000 (97.0000)  loss_ce_1_unscaled: 1.3571 (1.5956) 

...

Test:  [2000/2184]  eta: 0:00:14  class_error: 0.00  loss: 13.1819 (25.0933)  loss_ce: 1.8528 (1.6478)  loss_bbox: 0.0000 (1.4415)  loss_giou: 0.0000 (1.1626)  loss_ce_0: 1.5399 (1.4003)  loss_bbox_0: 0.0000 (1.4791)  loss_giou_0: 0.0000 (1.1540)  loss_ce_1: 1.5425 (1.5039)  loss_bbox_1: 0.0000 (1.4420)  loss_giou_1: 0.0000 (1.1553)  loss_ce_2: 1.7187 (1.5798)  loss_bbox_2: 0.0000 (1.4510)  loss_giou_2: 0.0000 (1.1587)  loss_ce_3: 1.8969 (1.6690)  loss_bbox_3: 0.0000 (1.4406)  loss_giou_3: 0.0000 (1.1550)  loss_ce_4: 1.9070 (1.6630)  loss_bbox_4: 0.0000 (1.4335)  loss_giou_4: 0.0000 (1.1563)  loss_ce_unscaled: 1.8528 (1.6478)  class_error_unscaled: 100.0000 (49.1754)  loss_bbox_unscaled: 0.0000 (0.2883)  loss_giou_unscaled: 0.0000 (0.5813)  cardinality_error_unscaled: 100.0000 (95.5435)  loss_ce_0_unscaled: 1.5399 (1.4003)  loss_bbox_0_unscaled: 0.0000 (0.2958)  loss_giou_0_unscaled: 0.0000 (0.5770)  cardinality_error_0_unscaled: 100.0000 (95.5447)  loss_ce_1_unscaled: 1.5425 (1.5039) 

Test:  [2050/2184]  eta: 0:00:10  class_error: 100.00  loss: 11.5061 (25.2071)  loss_ce: 1.8100 (1.6399)  loss_bbox: 0.0000 (1.4544)  loss_giou: 0.0000 (1.1767)  loss_ce_0: 1.5135 (1.3936)  loss_bbox_0: 0.0000 (1.4915)  loss_giou_0: 0.0000 (1.1671)  loss_ce_1: 1.6025 (1.4966)  loss_bbox_1: 0.0000 (1.4553)  loss_giou_1: 0.0000 (1.1686)  loss_ce_2: 1.7114 (1.5722)  loss_bbox_2: 0.0000 (1.4636)  loss_giou_2: 0.0000 (1.1721)  loss_ce_3: 1.8303 (1.6613)  loss_bbox_3: 0.0000 (1.4533)  loss_giou_3: 0.0000 (1.1693)  loss_ce_4: 1.8230 (1.6551)  loss_bbox_4: 0.0000 (1.4461)  loss_giou_4: 0.0000 (1.1703)  loss_ce_unscaled: 1.8100 (1.6399)  class_error_unscaled: 100.0000 (48.5129)  loss_bbox_unscaled: 0.0000 (0.2909)  loss_giou_unscaled: 0.0000 (0.5883)  cardinality_error_unscaled: 100.0000 (95.3237)  loss_ce_0_unscaled: 1.5135 (1.3936)  loss_bbox_0_unscaled: 0.0000 (0.2983)  loss_giou_0_unscaled: 0.0000 (0.5836)  cardinality_error_0_unscaled: 100.0000 (95.3250)  loss_ce_1_unscaled: 1.6025 (1.4966

Test:  [2100/2184]  eta: 0:00:06  class_error: 0.00  loss: 31.0948 (25.0014)  loss_ce: 1.6492 (1.6438)  loss_bbox: 1.6608 (1.4326)  loss_giou: 1.9026 (1.1609)  loss_ce_0: 1.4233 (1.3963)  loss_bbox_0: 1.8626 (1.4690)  loss_giou_0: 1.9369 (1.1517)  loss_ce_1: 1.4891 (1.5002)  loss_bbox_1: 1.9158 (1.4339)  loss_giou_1: 1.8999 (1.1530)  loss_ce_2: 1.5729 (1.5750)  loss_bbox_2: 1.8922 (1.4417)  loss_giou_2: 1.8926 (1.1562)  loss_ce_3: 1.6508 (1.6644)  loss_bbox_3: 1.7643 (1.4313)  loss_giou_3: 1.8673 (1.1537)  loss_ce_4: 1.6686 (1.6588)  loss_bbox_4: 1.6438 (1.4245)  loss_giou_4: 1.8986 (1.1545)  loss_ce_unscaled: 1.6492 (1.6438)  class_error_unscaled: 0.0000 (49.1195)  loss_bbox_unscaled: 0.3322 (0.2865)  loss_giou_unscaled: 0.9513 (0.5804)  cardinality_error_unscaled: 98.0000 (95.4200)  loss_ce_0_unscaled: 1.4233 (1.3963)  loss_bbox_0_unscaled: 0.3725 (0.2938)  loss_giou_0_unscaled: 0.9685 (0.5758)  cardinality_error_0_unscaled: 98.0000 (95.4212)  loss_ce_1_unscaled: 1.4891 (1.5002)  los

Test:  [2150/2184]  eta: 0:00:02  class_error: 0.00  loss: 40.5390 (25.0221)  loss_ce: 1.6318 (1.6482)  loss_bbox: 3.2285 (1.4339)  loss_giou: 1.9904 (1.1584)  loss_ce_0: 1.4227 (1.3992)  loss_bbox_0: 3.0011 (1.4701)  loss_giou_0: 1.9924 (1.1504)  loss_ce_1: 1.5566 (1.5043)  loss_bbox_1: 3.1601 (1.4362)  loss_giou_1: 2.0109 (1.1519)  loss_ce_2: 1.6002 (1.5791)  loss_bbox_2: 3.2894 (1.4431)  loss_giou_2: 1.9893 (1.1544)  loss_ce_3: 1.6114 (1.6685)  loss_bbox_3: 3.2673 (1.4324)  loss_giou_3: 1.9901 (1.1516)  loss_ce_4: 1.5737 (1.6630)  loss_bbox_4: 3.1854 (1.4252)  loss_giou_4: 1.9900 (1.1522)  loss_ce_unscaled: 1.6318 (1.6482)  class_error_unscaled: 0.0000 (49.0934)  loss_bbox_unscaled: 0.6457 (0.2868)  loss_giou_unscaled: 0.9952 (0.5792)  cardinality_error_unscaled: 99.0000 (95.5079)  loss_ce_0_unscaled: 1.4227 (1.3992)  loss_bbox_0_unscaled: 0.6002 (0.2940)  loss_giou_0_unscaled: 0.9962 (0.5752)  cardinality_error_0_unscaled: 99.0000 (95.5088)  loss_ce_1_unscaled: 1.5566 (1.5043)  los

Test: Total time: 0:02:53 (0.0792 s / it)
Averaged stats: class_error: 100.00  loss: 12.6252 (24.9505)  loss_ce: 2.2457 (1.6540)  loss_bbox: 0.0000 (1.4255)  loss_giou: 0.0000 (1.1493)  loss_ce_0: 1.7745 (1.4031)  loss_bbox_0: 0.0000 (1.4607)  loss_giou_0: 0.0000 (1.1425)  loss_ce_1: 1.9491 (1.5088)  loss_bbox_1: 0.0000 (1.4275)  loss_giou_1: 0.0000 (1.1440)  loss_ce_2: 2.1069 (1.5844)  loss_bbox_2: 0.0000 (1.4342)  loss_giou_2: 0.0000 (1.1462)  loss_ce_3: 2.2466 (1.6745)  loss_bbox_3: 0.0000 (1.4234)  loss_giou_3: 0.0000 (1.1433)  loss_ce_4: 2.2865 (1.6690)  loss_bbox_4: 0.0000 (1.4168)  loss_giou_4: 0.0000 (1.1433)  loss_ce_unscaled: 2.2457 (1.6540)  class_error_unscaled: 100.0000 (49.4505)  loss_bbox_unscaled: 0.0000 (0.2851)  loss_giou_unscaled: 0.0000 (0.5746)  cardinality_error_unscaled: 100.0000 (95.5717)  loss_ce_0_unscaled: 1.7745 (1.4031)  loss_bbox_0_unscaled: 0.0000 (0.2921)  loss_giou_0_unscaled: 0.0000 (0.5712)  cardinality_error_0_unscaled: 100.0000 (95.5726)  loss_ce_1_

In [1]:
import numpy as np
print(np.__version__)
np.float

1.23.5


/tmp/ipykernel_3486748/492108953.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.float


float

In [16]:
import os.path
from typing import Any, Callable, List, Optional, Tuple
from PIL import Image
from torchvision.datasets.vision import VisionDataset

class CocoDetection(VisionDataset):
    """`MS Coco Detection <https://cocodataset.org/#detection-2016>`_ Dataset.

    It requires the `COCO API to be installed <https://github.com/pdollar/coco/tree/master/PythonAPI>`_.

    Args:
        root (string): Root directory where images are downloaded to.
        annFile (string): Path to json annotation file.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.PILToTensor``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.
    """

    def __init__(
        self,
        root: str,
        annFile: str,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        transforms: Optional[Callable] = None,
    ) -> None:
        super().__init__(root, transforms, transform, target_transform)
        from pycocotools.coco import COCO

        self.coco = COCO(annFile)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def _load_image(self, id: int) -> Image.Image:
        path = self.coco.loadImgs(id)[0]["file_name"]
        return Image.open(os.path.join(self.root, path)).convert("RGB")

    def _load_target(self, id: int) -> List[Any]:
        return self.coco.loadAnns(self.coco.getAnnIds(id))

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        id = self.ids[index]
        image = self._load_image(id)
        target = self._load_target(id)

        if self.transforms is not None:
            image, target = self.transforms(image, target)

        return image, target


    def __len__(self) -> int:
        return len(self.ids)

In [94]:
!ls $dataDir/annotations

detr_test.json	detr_train.json


In [15]:
from pycocotools.coco import COCO

annFile = os.path.join(dataDir,'annotations','detr_test.json')
annFile = os.path.join(dataDir,'annotations','detr_train.json')
coco = COCO(annFile)
# ids = list(sorted(coco.imgs.keys()))

NameError: name 'os' is not defined

In [130]:
with open(annFile, 'r') as f:
    coco = json.load(f)
    
coco['images'] = sorted(coco['images'], key=lambda x: x['image_id']) 
coco['annotations'] = sorted(coco['annotations'], key=lambda x: x['image_id']) 

count = 0
for idx in range(len(coco['images'])):
    coco['images'][idx]['id'] = coco['images'][idx]['image_id']
    coco['annotations'][idx]['id'] = coco['annotations'][idx]['image_id']
    seg_info = coco['annotations'][idx]['segments_info']
    if (len(seg_info) ==1)&(len(seg_info[0]['bbox'])==0):
        coco['annotations'][idx]['segments_info'][0]['category_id'] = 0
        coco['annotations'][idx]['category_id']=0
        count = count + 1
    else:
        coco['annotations'][idx]['category_id']=1

print("number of normal images:", count/48)

number of normal images: 458.0


In [131]:
coco['annotations'][0]

{'image_id': 0,
 'file_name': '0.npy',
 'frame_idx': 0,
 'segments_info': [{'area': 126,
   'iscrowd': 0,
   'bbox': [301, 143, 22, 9],
   'category_id': 1,
   'id': 0},
  {'area': 35,
   'iscrowd': 0,
   'bbox': [212, 148, 10, 4],
   'category_id': 1,
   'id': 1},
  {'area': 10,
   'iscrowd': 0,
   'bbox': [253, 189, 4, 3],
   'category_id': 1,
   'id': 2},
  {'area': 17,
   'iscrowd': 0,
   'bbox': [207, 201, 5, 4],
   'category_id': 1,
   'id': 3},
  {'area': 29,
   'iscrowd': 0,
   'bbox': [205, 208, 5, 7],
   'category_id': 1,
   'id': 4}],
 'id': 0,
 'category_id': 1}

In [132]:
coco.keys()

dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])

In [133]:
coco['images'][0]

{'image_id': 0,
 'file_name': '0.npy',
 'frame_idx': 0,
 'width': 566,
 'height': 566,
 'age': '063Y',
 'sex': 'F',
 'diagnosis': 'MELANOMA',
 'nii_file_name:': '/gpfs/fs0/data/stanford_data/petct/NIFTI_MIP/FDG-PET-CT-Lesions/PETCT_0011f3deaf/03-23-2003-NA-PET-CT Ganzkoerper  primaer mit KM-10445/SUV_MIP.nii.gz',
 'nii_stats': {'liver': 4.024835109710693,
  'brain': 2.4035794734954834,
  'suv_max': 25.339075088500977,
  'pixdim': [-1.0,
   2.0364201068878174,
   2.0364201068878174,
   3.0,
   1.0,
   1.0,
   1.0,
   1.0]},
 'id': 0}

In [135]:
print(len(coco['images']))
print(len(coco['annotations']))

annFile_test2 = os.path.join(dataDir,'annotations','detr_test2.json')
annFile_train2 = os.path.join(dataDir,'annotations','detr_train2.json')
print(annFile_test2)
print(annFile_train2)

43344
43344
/gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/annotations/detr_test2.json
/gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/annotations/detr_train2.json


In [14]:
import json
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)   

# with open(annFile_train2,"w") as outfile:
#     json.dump(coco, outfile, cls=NpEncoder)

In [179]:
def get_annotations(ann_file):
    with open(ann_file, 'r') as f:
        coco = json.load(f)
    
    # sort 'images' and 'annotations' so that they are aligned with 'annotations'
    # i.e., in alphabetical order
    coco['images'] = sorted(coco['images'], key=lambda x: x['image_id']) 
    coco['annotations'] = sorted(coco['annotations'], key=lambda x: x['image_id']) 
    
    return coco


def train_val_split(ann_file, train_split, cross_val, val_fold):
    coco = get_annotations(ann_file)
    
    train_dict = {'images':[],'annotations':[],'info':coco['info'], 'licenses':coco['licenses'], 'categories':coco['categories']}
    val_dict = {'images':[],'annotations':[],'info':coco['info'], 'licenses':coco['licenses'], 'categories':coco['categories']}
    num_train = math.floor((train_split*len(coco['images']))//48)*48 #there are 48 slices per study
    
    # TO DO: If NOT cross validation, then split per train_split
    if not cross_val:
        train_dict['images'] = coco['images'][:num_train].copy()
        train_dict['annotations'] = coco['annotations'][:num_train].copy()
        val_dict['images'] = coco['images'][num_train:].copy()
        val_dict['annotations'] = coco['annotations'][num_train:].copy()
    # If cross validation, sort by image_id 
    else:
        num_folds = 1//(1-train_split)
        train_folds = list(range(0,num_folds)).remove(val_fold)
        num_val = len(coco['images'])-num_train
        coco['images'] = [coco['images'][i:i+num_val] for i in range(0, len(coco['images']), num_val)]
        coco['annotations'] = [coco['annotations'][i:i+num_val] for i in range(0, len(coco['annotations']), num_val)]
        val_dict['images'] = coco['images'][val_fold].copy()
        val_dict['annotations'] = coco['annotations'][val_fold].copy()
        for fold in train_folds:
            train_dict['images'] = train_dict['images'].extend(coco['images'][fold].copy())
            train_dict['annotations'] = train_dict['annotations'].extend(coco['annotations'][fold].copy())
    
    # shuffle the images -- otherwise 48 images in a row will be from same patient -- coco pipeline does this downstream already
    #train_dict['images'], train_dict['annotations'] = random_shuffle_two_lists(train_dict['images'], train_dict['annotations'])
    
    return train_dict, val_dict


def write_json(coco, out_path):
    with open(out_path,"w") as outfile:
        json.dump(coco, outfile, cls=NpEncoder)
        

In [205]:
# if not os.path.isdir('/gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/'):
#     os.makedirs('/gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/development')
!ls /gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/annotations

ann_file = '/gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/annotations/detr_dev_all.json'
train_dict, val_dict = train_val_split(ann_file, 0.9, False, 1)

detr_dev_all.json  detr_test.json  detr_train_0.9.json	detr_val_0.1.json


In [206]:
train_dict['images'][0]

{'image_id': 0,
 'file_name': '0.npy',
 'frame_idx': 0,
 'width': 566,
 'height': 566,
 'age': '063Y',
 'sex': 'F',
 'diagnosis': 'MELANOMA',
 'nii_file_name:': '/gpfs/fs0/data/stanford_data/petct/NIFTI_MIP/FDG-PET-CT-Lesions/PETCT_0011f3deaf/03-23-2003-NA-PET-CT Ganzkoerper  primaer mit KM-10445/SUV_MIP.nii.gz',
 'nii_stats': {'liver': 4.024835109710693,
  'brain': 2.4035794734954834,
  'suv_max': 25.339075088500977,
  'pixdim': [-1.0,
   2.0364201068878174,
   2.0364201068878174,
   3.0,
   1.0,
   1.0,
   1.0,
   1.0]},
 'id': 0}

In [207]:
out_path_train = '/gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/annotations/detr_train_0.9.json'
out_path_val = '/gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/annotations/detr_val_0.1.json'

write_json(train_dict, out_path_train)
write_json(val_dict, out_path_val)

In [240]:
with open(out_path_train, 'r') as f:
    coco2 = json.load(f)
    
coco2['annotations'][1]

{'image_id': 1,
 'file_name': '1.npy',
 'frame_idx': 1,
 'segments_info': [{'area': 134,
   'iscrowd': 0,
   'bbox': [290, 143, 23, 9],
   'category_id': 1,
   'id': 5},
  {'area': 35,
   'iscrowd': 0,
   'bbox': [203, 148, 10, 4],
   'category_id': 1,
   'id': 6},
  {'area': 10,
   'iscrowd': 0,
   'bbox': [255, 189, 4, 3],
   'category_id': 1,
   'id': 7},
  {'area': 17,
   'iscrowd': 0,
   'bbox': [203, 201, 5, 4],
   'category_id': 1,
   'id': 8},
  {'area': 29,
   'iscrowd': 0,
   'bbox': [203, 208, 5, 7],
   'category_id': 1,
   'id': 9}],
 'id': 1,
 'category_id': 1}

In [227]:
coco2.keys()

dict_keys(['images', 'annotations', 'info', 'licenses', 'categories'])

In [17]:
coco3 = COCO('/gpfs/fs0/data/stanford_data/petct/DETR_MIP2/FDG-PET-CT-Lesions/annotations/detr_val.json')

loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


In [18]:
coco3.getImgIds(43340)

[43340]

In [19]:
coco3.imgs[43340]

{'id': 43340,
 'file_name': '43340.npy',
 'frame_idx': 44,
 'width': 566,
 'height': 566,
 'age': '057Y',
 'sex': 'F',
 'diagnosis': 'NEGATIVE',
 'nii_file_name:': '/gpfs/fs0/data/stanford_data/petct/NIFTI_MIP/FDG-PET-CT-Lesions/PETCT_e1a5b05186/02-27-2005-NA-PET-CT Ganzkoerper  primaer mit KM-36173/SUV_MIP.nii.gz',
 'nii_stats': {'liver': 12.701639175415039,
  'brain': 37.47007751464844,
  'suv_max': 37.47007751464844,
  'pixdim': [-1.0,
   2.0364201068878174,
   2.0364201068878174,
   3.0,
   1.0,
   1.0,
   1.0,
   1.0]}}

In [20]:
coco3.loadImgs(coco3.getImgIds(43340))

[{'id': 43340,
  'file_name': '43340.npy',
  'frame_idx': 44,
  'width': 566,
  'height': 566,
  'age': '057Y',
  'sex': 'F',
  'diagnosis': 'NEGATIVE',
  'nii_file_name:': '/gpfs/fs0/data/stanford_data/petct/NIFTI_MIP/FDG-PET-CT-Lesions/PETCT_e1a5b05186/02-27-2005-NA-PET-CT Ganzkoerper  primaer mit KM-36173/SUV_MIP.nii.gz',
  'nii_stats': {'liver': 12.701639175415039,
   'brain': 37.47007751464844,
   'suv_max': 37.47007751464844,
   'pixdim': [-1.0,
    2.0364201068878174,
    2.0364201068878174,
    3.0,
    1.0,
    1.0,
    1.0,
    1.0]}}]

In [21]:
# coco3.imgs[0]
coco3.loadImgs(coco3.getImgIds(43340))[0]

{'id': 43340,
 'file_name': '43340.npy',
 'frame_idx': 44,
 'width': 566,
 'height': 566,
 'age': '057Y',
 'sex': 'F',
 'diagnosis': 'NEGATIVE',
 'nii_file_name:': '/gpfs/fs0/data/stanford_data/petct/NIFTI_MIP/FDG-PET-CT-Lesions/PETCT_e1a5b05186/02-27-2005-NA-PET-CT Ganzkoerper  primaer mit KM-36173/SUV_MIP.nii.gz',
 'nii_stats': {'liver': 12.701639175415039,
  'brain': 37.47007751464844,
  'suv_max': 37.47007751464844,
  'pixdim': [-1.0,
   2.0364201068878174,
   2.0364201068878174,
   3.0,
   1.0,
   1.0,
   1.0,
   1.0]}}

In [32]:
coco3.anns[186211]

{'image_id': 38976,
 'id': 186211,
 'segmentation': {'counts': [156880,
   1,
   1,
   1,
   563,
   4,
   561,
   5,
   561,
   5,
   561,
   5,
   561,
   5,
   561,
   5,
   562,
   1,
   1,
   2,
   159510],
  'size': [566, 566]},
 'area': 34,
 'iscrowd': 0,
 'bbox': [277, 97, 8, 5],
 'category_id': 1}

In [33]:
coco3.loadAnns(coco3.getAnnIds(186211))

[]

In [274]:
coco['images'][0]

{'image_id': 0,
 'file_name': '0.npy',
 'frame_idx': 0,
 'width': 566,
 'height': 566,
 'age': '063Y',
 'sex': 'F',
 'diagnosis': 'MELANOMA',
 'nii_file_name:': '/gpfs/fs0/data/stanford_data/petct/NIFTI_MIP/FDG-PET-CT-Lesions/PETCT_0011f3deaf/03-23-2003-NA-PET-CT Ganzkoerper  primaer mit KM-10445/SUV_MIP.nii.gz',
 'nii_stats': {'liver': 4.024835109710693,
  'brain': 2.4035794734954834,
  'suv_max': 25.339075088500977,
  'pixdim': [-1.0,
   2.0364201068878174,
   2.0364201068878174,
   3.0,
   1.0,
   1.0,
   1.0,
   1.0]},
 'id': 0}

In [272]:
boxes=[[1,1,2,2],[1,1,2,1]]
boxes = torch.as_tensor(boxes, dtype=torch.float32)
print(boxes, boxes.shape)
boxes = boxes.reshape(-1, 4)
print(boxes, boxes.shape)
boxes[:, 2:] += boxes[:, :2]
print(boxes)
boxes[:, 0::2].clamp_(min=0, max=4)
print(boxes)
boxes[:, 1::2].clamp_(min=0, max=5)
print(boxes)

tensor([[1., 1., 2., 2.],
        [1., 1., 2., 1.]]) torch.Size([2, 4])
tensor([[1., 1., 2., 2.],
        [1., 1., 2., 1.]]) torch.Size([2, 4])
tensor([[1., 1., 3., 3.],
        [1., 1., 3., 2.]])
tensor([[1., 1., 3., 3.],
        [1., 1., 3., 2.]])
tensor([[1., 1., 3., 3.],
        [1., 1., 3., 2.]])


In [342]:
print(type(coco3))

<class 'pycocotools.coco.COCO'>


In [ ]:
class CocoDetection(VisionDataset):
    """`MS Coco Detection <https://cocodataset.org/#detection-2016>`_ Dataset.

    It requires the `COCO API to be installed <https://github.com/pdollar/coco/tree/master/PythonAPI>`_.

    Args:
        root (string): Root directory where images are downloaded to.
        annFile (string): Path to json annotation file.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.PILToTensor``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.
    """

    def __init__(
        self,
        root: str,
        annFile: str,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        transforms: Optional[Callable] = None,
    ) -> None:
        super().__init__(root, transforms, transform, target_transform)
        from pycocotools.coco import COCO

        self.coco = COCO(annFile)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def _load_image(self, id: int) -> Image.Image:
        path = self.coco.loadImgs(id)[0]["file_name"]
        return Image.open(os.path.join(self.root, path)).convert("RGB")

    def _load_target(self, id: int) -> List[Any]:
        return self.coco.loadAnns(self.coco.getAnnIds(id))

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        id = self.ids[index]
        image = self._load_image(id)
        target = self._load_target(id)

        if self.transforms is not None:
            image, target = self.transforms(image, target)

        return image, target


    def __len__(self) -> int:
        return len(self.ids)

# Results

Quick and easy overview of the training results

In [ ]:
from util.plot_utils import plot_logs

from pathlib import Path

log_directory = [Path(outDir)]

In [ ]:
fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'loss_ce',
    'loss_bbox',
    'loss_giou',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(log_directory,
          fields_of_interest)   